# Know Error
🟩 The bot tends to genrate multiple responese for a single reply (fixed)

🟩 Genrates multiple responses while working with media files (fixed)

🟩 Is able to remember images only time it is uploaded.(fixed)

🟩 Improving the video responses(Direct function added by google for video file)

🟩 Finding a way to keep the converstion for the videos conversation

🟩 If the user sends a gif through the inbuilt gif section, it's html page is downloaded.(fixed)

[The gif is converted to a mp4 file. If your gif is shorter then it will treat it as a still photo as gemini sees videos with one frame per seconds, which might lead to wrong interpretation of the gif]

🟩 Gifs from the discord keyboard(tenor), doesn't get interpretted correct(because different gifs get donwloaded) (fixed)

 !check_token doesn't  work for direct messages

# Goal
1. Fix the 2000 char limit. ✅
2. Saving the chats. ✅
3. Only reply when tagged or replied ✅
4. Consinent chats ✅
5. Image input ✅
6. Mutimedia input ✅
7. Webhook ✅

# d_bot
This notebook lets you run a discord bot powered by google gemini api, with conversation memory which includes audio, video, and images.

You can type **"!check_token"** to check how many tokens are being used.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1OQGPc2CsYpnBhNfKEl0O0cDT6uXAcQ8g?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  
</table>


In [ ]:
#@title <b>v-- If you play on mobile, tap this to open music player and play the white noise to keep tab running in the background. or your session might get disconnected
%%html
<b>Press play on the music player to keep the tab alive, then start block below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [1]:
# @title Mount your google drive
from google.colab import drive
from IPython.display import clear_output
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Install requirments

In [2]:
!pip install -q -U google-generativeai
!pip install Discord
!pip install python-magic
!pip install nest_asyncio
!pip install moviepy
!pip install pytz
clear_output()
print("Success")

Success


Step 2: Get the api key

In [3]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# @title Step 2.5: List available models
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

print("Now select any one of the model and paste it in the 'model_name' below")

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827
Now select any one of the model and paste it in the 'model_name' below


In [4]:
# Set the event listener for the dropdown change
# @title Model configuration
text_generation_config = {
    "temperature": 1.35,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

# Initial prompt
system_instruction = "Remember that you have the power of python to solve logical question if possible, don't forgot to try. When you the see the user message in the following format = ([string], [number]): {message content}. It means the conversation is happening in a server in discord. The string represents the username of the of the user who have sent the message and the number is the user id of the user.  Multiple people can interact during this, make sure too act accordingly. If you don't see this format and just see this format = (number) it means they are talking to you in dm, so act accordingly." # @param {type:"string"}
model_name = "models/gemini-1.5-pro-latest" # @param {type:"string"}
webhook_custom_prompt = "e" # @param {type:"string"}

# Create the model using the selected model name from the dropdown
model = genai.GenerativeModel(model_name = model_name, generation_config=text_generation_config, system_instruction=system_instruction, safety_settings=safety_settings, tools="code_execution")

In [5]:
# @title Functions
#Rembeer not to copy the timezone code
import discord
from discord import app_commands,Message
import time
from random import choice, randint
import os
import requests
from urllib.parse import urlparse, unquote
import re
import cv2
import shutil
import mimetypes
import magic
import json
import pickle
from datetime import datetime, timedelta
import moviepy.editor as mp
from bs4 import BeautifulSoup
from google.api_core.exceptions import GoogleAPIError
import google.generativeai as genai

def upload_to_gemini(path, mime_type=None):
    """Uploads the given file to Gemini."""
    file = genai.upload_file(path, mime_type=mime_type)
    print(f"Uploaded file '{file.display_name}' as: {file.uri}")
    return file

def wait_for_files_active(files):
    """Waits for the given files to be active."""
    print("Waiting for file processing...")
    for name in (file.name for file in files):
        file = genai.get_file(name)
        while file.state.name == "PROCESSING":
            print(".", end="", flush=True)
            time.sleep(10)
            file = genai.get_file(name)
        if file.state.name != "ACTIVE":
            raise Exception(f"File {file.name} failed to process")
    print("...all files ready")
    print()

def download_file(attachment_link: str, user_id: str) -> tuple:
    """
    Downloads the file, determines its type, and renames it with the correct extension.
    """
    try:
        # Parse the URL to get the file name without the extension
        parsed_link = urlparse(unquote(attachment_link))
        path = parsed_link.path
        original_filename = os.path.basename(path).split('?')[0]

        # Create a temporary filename with the user_id as prefix and no extension
        temp_filename = f'file_{user_id}'
        temp_filename_no_ext = temp_filename.rsplit('.', 1)[0]

        # Download the file
        response = requests.get(attachment_link)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Save the file without an extension
        with open(temp_filename_no_ext, 'wb') as f:
            f.write(response.content)

        print(f"File downloaded successfully: {temp_filename_no_ext}")

        # Determine the file type and the correct extension
        mime_type = determine_file_type(temp_filename_no_ext)
        extension = mimetypes.guess_extension(mime_type) or '.bin'  # Default to .bin if unknown

        # Check if the downloaded file is actually a GIF
        if mime_type == 'text/html' and 'tenor.com' in parsed_link.netloc:
            media_url = extract_media_url_from_html(temp_filename_no_ext)
            if media_url:
                os.remove(temp_filename_no_ext)
                return download_file(media_url, user_id)
            else:
                raise ValueError("Unable to extract media URL from Tenor HTML")

        # Rename the file with the correct extension
        final_filename = f'{temp_filename_no_ext}{extension}'
        os.rename(temp_filename_no_ext, final_filename)
        print(f"File renamed to: {final_filename} with MIME type: {mime_type}")

        return mime_type, final_filename
    except requests.RequestException as e:
        print(f"Failed to download file. Error: {e}")
        return
    except Exception as e:
        print(f"An error occurred: {e}")
        return

def determine_file_type(filepath: str) -> str:
    """
    Determines the MIME type of a file by reading its contents.
    """
    try:
        mime = magic.Magic(mime=True)
        mime_type = mime.from_file(filepath)
        return mime_type
    except Exception as e:
        print(f"Could not determine file type. Error: {e}")
        return 'application/octet-stream'

# Function to load chat history from a file
def load_chat_history(user_id, custom_file_path, bot_id):
    full_path = os.path.join(custom_file_path, f'{bot_id}_{user_id}_chat_history.pkl')
    os.makedirs(custom_file_path, exist_ok=True)

    if not os.path.exists(full_path):
        with open(full_path, 'wb') as file:
            pickle.dump([], file)

    with open(full_path, 'rb') as file:
        chat_history = pickle.load(file)

    return chat_history

# Function to save the chat history from a file
def save_chat_history(user_id, chat, custom_file_path, bot_id):
    full_path = os.path.join(custom_file_path, f'{bot_id}_{user_id}_chat_history.pkl')  # Use os.path.join
    with open(full_path, 'wb') as file:
        pickle.dump(chat.history, file)

def save_filetwo(user_id, time_file_path, url, bot_id):
    file_name = f'{bot_id}_{user_id}_files_metadata.json'
    file_path = os.path.join(time_file_path, file_name)

    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        with open(file_path, 'w') as file:
            json.dump([], file)

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []

    new_data = {
        'file_uri': url,
        'timestamp': datetime.now().isoformat()
    }
    data.append(new_data)
    print(file_path)

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
        print("Successful saved the file url and upload time")

def check_expired_files(user_id, time_file_path, history, bot_id):
    tempoery = []
    chat_history = history
    file_path = os.path.join(time_file_path, f'{bot_id}_{user_id}_files_metadata.json')

    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            json.dump([], file)
        return history

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []

    current_time = datetime.utcnow()
    expired_files = []

    for entry in data:
        upload_time = datetime.fromisoformat(entry['timestamp'])
        if current_time - upload_time > timedelta(hours=48):
            expired_files.append(entry)

        for dct in expired_files:
            tempoery.append(dct['file_uri'])

        for link in tempoery:
            target_word = (f'{link}')
            chat_history = [entry for entry in chat_history if target_word not in str(entry)]
            print(f'Successfully removed: {target_word}')

            data = [entry for entry in data if entry['file_uri'] != target_word]

            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
                print("Successfully updated the file_metadata.json")

    return chat_history

def extract_media_url_from_html(html_file_path):
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    media_url = None
    for meta in soup.find_all('meta'):
        if meta.get('name') == 'twitter:player:stream':
            media_url = meta.get('content')
            break

    return media_url

async def send_processing_notice(message: Message) -> None:  # Define the function
    """Sends the "Processing message" notice after a short delay."""
    #await asyncio.sleep(0.0001)  # Adjust delay as needed
    await message.channel.send(f"<@{(message.author.id)}> ⚠️ The bot is currently processing another request. Please wait a moment.")

class GeminiErrorHandler:
    def __init__(self):
        self.error_messages = {
            400: "⚠️ Error 400: Invalid Argument. Please check your request format and parameters. Please if eariler you were using the model 'gemini 1.0 pro' and then switched to any model for gemini 1.5 it will cause error.",
            403: "⚠️ Error 403: Permission Denied. Check your API key and authentication.",
            404: "⚠️ Error 404: Not Found. The requested resource was not found.",
            429: "⚠️ Error 429: Too Many Requests. Please try again later.",
            500: "⚠️ Error 500: Internal Server Error. Please try again later or reduce the input context.",
            503: "⚠️ Error 503: Service Unavailable. Please try again later or switch to a different model.",
            504: "⚠️ Error 504: Deadline Exceeded. Please try again later or set a larger timeout."
        }

    async def handle_error(self, message: Message, error: GoogleAPIError, id_str: str):
        error_code = error.code if hasattr(error, "code") else 500
        error_message = self.error_messages.get(
            error_code,
            f"⚠️ An unexpected error occurred (code {error_code}). Please try again later."
        )

        if message:  # Check if the message object is available
            await send_message_main_bot(message, error_message)  # Use the main bot function
        else:
            print(f"Error sending error message: No message object available.")  # Log the error

        print(f"Error: {error_message} (Details: {error})")

def load_webhook_system_instruction(webhook_id: str, webhooks_path=None) -> str:
    json_file = os.path.join(webhooks_path, "webhooks_data.json")
    if os.path.exists(json_file):
        with open(json_file, "r") as f:
            webhooks_dict = json.load(f)

        webhook_data = webhooks_dict.get(webhook_id)
        if webhook_data:
            return webhook_data.get("system_instructions", "")

    # If we can't find the specific webhook instruction, return a default
    return "You are a helpful assistant."

async def send_message_main_bot(message: Message, response: str) -> None:
    """Sends a message to the channel where the original message was received."""

    print("Sending message via main bot...")
    destination = message.channel

    if len(response) <= 2000:
        await destination.send(response)
    else:
        chunks = re.findall(r".{1,2000}(?:\s|$)", response, re.DOTALL)
        for chunk in chunks:
            chunk = chunk.strip()
            if chunk:
                await destination.send(chunk)


async def send_message_webhook(webhook: discord.Webhook, response: str) -> None:
    """Sends a message using the specified webhook."""

    print("Sending message via webhook...")
    destination = webhook

    if len(response) <= 2000:
        await destination.send(response)
    else:
        chunks = re.findall(r".{1,2000}(?:\s|$)", response, re.DOTALL)
        for chunk in chunks:
            chunk = chunk.strip()
            if chunk:
                await destination.send(chunk)

# Create an instance of the error handler
error_handler = GeminiErrorHandler()

# Running The Bot

In [ ]:
import re
import aiohttp
from typing import Final, Dict
import os
import discord
from discord import Intents, Client, Message, app_commands, WebhookMessage
from discord.ext import commands
from google.colab import userdata
import PIL.Image
import nest_asyncio
from datetime import datetime
from google.ai.generativelanguage_v1beta.types import content
import google.generativeai as genai
from IPython.display import HTML, display
import pytz
import asyncio
import io
import base64
from google.ai.generativelanguage_v1beta.types import content
import json

# STEP 0: LOAD OUR TOKEN FROM SOMEWHERE SAFE
TOKEN: Final[str] = userdata.get('DISCORD_TOKEN')
custom_path = '/content/drive/MyDrive/Discord_bot/'
time_file_path = '/content/drive/MyDrive/Discord_bot/Time_files/'
webhooks_path = '/content/drive/MyDrive/Discord_bot/webhooks/'
#chat_instances = []

# STEP 1: BOT SETUP
intents: Intents = Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix='!', intents=intents, application_id=1228578114582482955)
processing_messages = {}
webhooks: Dict[int, discord.Webhook] = {}

# STEP 2: MESSAGE FUNCTIONALITY

@bot.tree.command(name="test", description="A simple test command")
async def test_command(interaction: discord.Interaction):
    await interaction.response.send_message("Hello!", ephemeral=False)
    print("test command used!")

@bot.tree.command(name="check_token_usage", description="Check the token usage")
async def check_token_usage(interaction: discord.Interaction):
    await interaction.response.defer()  # Defer the response as this might take a while
    print("check_token_usage command used!")

    id_str = str(interaction.channel.id)
    bot_id = "main_bot_"

    history = load_chat_history(id_str, custom_path, bot_id)  # Use the updated load_chat_history
    chat_history = check_expired_files(id_str, time_file_path, history, bot_id)  # Use the updated check_expired_files
    chat = model.start_chat(history=chat_history)
    token_count = model.count_tokens(chat.history)

    response = f"{token_count}"
    await interaction.followup.send(response)


@bot.tree.command(name="info", description="Displays bot information")
async def info_command(interaction: discord.Interaction):
    await interaction.response.defer()  # Defer the response as it might take a bit

    # Get the bot's latency
    latency = bot.latency * 1000

    # Create an embed to display the information nicely
    embed = discord.Embed(title="Bot Information", color=discord.Color.blue())
    embed.add_field(name="Model Name", value=model_name, inline=False)
    embed.add_field(name="Ping", value=f"{latency:.2f} ms", inline=False)

    # Create a temporary text file with the system instructions
    with open("system_instructions.txt", "w") as f:
        f.write(system_instruction)

    # Send the embed and the text file as an attachment
    await interaction.followup.send(embed=embed, file=discord.File("system_instructions.txt"))

@bot.tree.command(name="add_webhook", description="Adds a webhook to the channel with system instructions")
@app_commands.describe(
    name="The name for the webhook",
    avatar="The avatar image for the webhook (png/jpg/webp, optional)",
    plain_text_instructions="System instructions as plain text (either this or text_file_instructions is required)",
    text_file_instructions="System instructions as a text file attachment (either this or plain_text_instructions is required)"
)
async def add_webhook_command(
    interaction: discord.Interaction,
    name: str,
    avatar: discord.Attachment = None,
    plain_text_instructions: str = None,
    text_file_instructions: discord.Attachment = None
):
    await interaction.response.defer()

    try:
        # Check if exactly one of plain_text_instructions or text_file_instructions is provided
        if (plain_text_instructions is None) == (text_file_instructions is None):
            await interaction.followup.send("Please provide either plain text instructions or a text file with instructions.")
            return

        # Get system instructions
        if plain_text_instructions:
            system_instructions = plain_text_instructions
        else:
            # Check if the attachment is a text file
            if not text_file_instructions.content_type.startswith("text/"):
                await interaction.followup.send("Invalid instructions file type. Please provide a text file.")
                return
            system_instructions = (await text_file_instructions.read()).decode("utf-8")

        # Download the avatar image (if provided)
        avatar_bytes = None
        if avatar:
            if avatar.content_type not in ["image/png", "image/jpeg", "image/webp"]:
                await interaction.followup.send("Invalid avatar file type. Please provide a png, jpg, or webp image.")
                return
            avatar_bytes = await avatar.read()

        # Create the webhook
        webhook = await interaction.channel.create_webhook(name=name, avatar=avatar_bytes)

        # Store the webhook
        webhooks[webhook.id] = webhook

        # Store webhook data (webhook's user_id and system instructions) in a JSON file
        webhook_data = {
            "webhook_user_id": webhook.id,  # Capturing the webhook's user_id
            "system_instructions": system_instructions
        }

        # Load existing webhooks from JSON (if exists)
        if not os.path.exists(webhooks_path):
            os.makedirs(webhooks_path)

        json_file = os.path.join(webhooks_path, "webhooks_data.json")
        if os.path.exists(json_file):
            with open(json_file, "r") as f:
                webhooks_dict = json.load(f)
        else:
            webhooks_dict = {}

        # Add the new webhook to the dictionary
        webhooks_dict[str(webhook.id)] = webhook_data

        # Save the updated dictionary back to the JSON file
        with open(json_file, "w") as f:
            json.dump(webhooks_dict, f, indent=4)

        await interaction.followup.send(f"Webhook '{name}' created successfully with system instructions!")
        await webhook.send("Hello! I'm ready with my instructions.")


    except discord.HTTPException as e:
        await interaction.followup.send(f"Error creating webhook: {e}")

@bot.tree.command(name="remove_webhook", description="Removes a webhook created by the bot")
@app_commands.describe(name="The name of the webhook to remove")
async def remove_webhook_command(interaction: discord.Interaction, name: str):
    await interaction.response.defer()

    try:
        # Get all webhooks in the channel
        webhooks = await interaction.channel.webhooks()

        # Filter out the webhooks created by the bot and match the name
        bot_webhooks = [webhook for webhook in webhooks if webhook.user == bot.user and webhook.name == name]

        if not bot_webhooks:
            await interaction.followup.send(f"No webhook named '{name}' created by the bot was found.")
            return

        # Load the webhook data from the JSON file
        json_file = os.path.join(webhooks_path, "webhooks_data.json")
        if os.path.exists(json_file):
            with open(json_file, "r") as f:
                webhooks_dict = json.load(f)
        else:
            webhooks_dict = {}

        # Delete each webhook that matches the criteria and remove from JSON file
        for webhook in bot_webhooks:
            await webhook.delete()

            # Remove the webhook from the dictionary
            if str(webhook.id) in webhooks_dict:
                del webhooks_dict[str(webhook.id)]

        # Save the updated dictionary back to the JSON file
        with open(json_file, "w") as f:
            json.dump(webhooks_dict, f, indent=4)

        await interaction.followup.send(f"Webhook '{name}' removed successfully!")

    except discord.HTTPException as e:
        await interaction.followup.send(f"Error removing webhook: {e}")

@bot.tree.command(name="add_v2_card_characters", description="Adds a V2 card character using a PNG file")
@app_commands.describe(
    image="The PNG image file containing the character data (required)"
)
async def add_v2_card_characters(
    interaction: discord.Interaction,
    image: discord.Attachment
):
    await interaction.response.defer()

    try:
        # Check if the attachment is a PNG file
        if not image.content_type == "image/png":
            await interaction.followup.send("Invalid image file type. Please provide a PNG image.")
            return

        # Download the image
        image_bytes = await image.read()

        # Open the image using PIL
        img = PIL.Image.open(io.BytesIO(image_bytes))

        # Extract the metadata
        meta_data = img.info

        # Extract and decode the base64 string
        base64_message = meta_data.get('chara')
        if base64_message:
            base64_bytes = base64_message.encode('ascii')
            message_bytes = base64.b64decode(base64_bytes)
            extracted_text = message_bytes.decode('ascii')
        else:
            await interaction.followup.send("No 'chara' metadata found in the image.")
            return
        try:
          data_dict = json.loads(extracted_text)
        except json.JSONDecodeError:
          print("Error: Invalid JSON string")

        character_data = data_dict['data']
        name = character_data['name']
        description = f"The description for {name} i.e. is given in the proceduing text" + (character_data['description']) + "The information about the description has ended!!"
        scenario = f"The current scenario for you i.e. {name} is " + (character_data['scenario']) + "The information about the scenario has ended!!"
        system_prompt = f"This is an interal instructions on how you have to genrated the responses" + (character_data['system_prompt']) + "The information about the system prompt has ended!!"
        message_example = f"These are some message example that you can refer to while genrating responses but it's important that you don't bound yourself to always follow the example you can also genrate a complete different repsons based on the situation. Here is the example:" + (character_data['mes_example']) + "The information about the message example has ended!!"

        name_ins = f'You are now tasked to roleplay as "{name}" and not google gemini. Further info about {name} is given after this text.'

        user_id = interaction.user.id
        greeting = character_data['first_mes']
        greeting = re.sub(r'{{user}}', f'<@{user_id}>', greeting)
        greeting = re.sub(r'{{char}}', f'{name}', greeting)
        processed_instructions = f"{system_prompt}\n{name_ins}\n{description}\n{scenario}\n{message_example}"

        # Create the webhook with the image as the avatar
        webhook = await interaction.channel.create_webhook(name=name, avatar=image_bytes)

        # Store the webhook
        webhooks[webhook.id] = webhook

        # Store webhook data (webhook's user_id and extracted text as system instructions) in a JSON file
        webhook_data = {
            "webhook_user_id": webhook.id,
            "system_instructions": processed_instructions
        }
        id_str = str(interaction.channel.id)
        bot_id = str(webhook.id)
        #model_variable = bot_id + id_str
        system_instruction = f"This is the main instrction that you have to follow 76741743:{webhook_custom_prompt}:76741743" + processed_instructions
        custom_model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=text_generation_config,
        system_instruction=system_instruction,
        safety_settings=safety_settings,
        tools="code_execution"
        )

        intial_prompt = [
          {
              "role": "model",
              "parts": [
                  {
                      "text": f"{greeting}"
                  }
              ]
          }
        ]

        chat = model.start_chat(history=intial_prompt)
        save_chat_history(id_str, chat, custom_path, bot_id)

        # Load existing webhooks from JSON (if exists)
        if not os.path.exists(webhooks_path):
            os.makedirs(webhooks_path)

        json_file = os.path.join(webhooks_path, "webhooks_data.json")
        if os.path.exists(json_file):
            with open(json_file, "r") as f:
                webhooks_dict = json.load(f)
        else:
            webhooks_dict = {}

        # Add the new webhook to the dictionary
        webhooks_dict[str(webhook.id)] = webhook_data

        # Save the updated dictionary back to the JSON file
        with open(json_file, "w") as f:
            json.dump(webhooks_dict, f, indent=4)

        await interaction.followup.send(f"Character '{name}' added successfully with extracted data as system instructions!")

        await send_message_webhook(webhook=webhook, response=greeting)

    except discord.HTTPException as e:
        await interaction.followup.send(f"Error adding character: {e}")
    except Exception as e:
        await interaction.followup.send(f"An error occurred: {e}")

@bot.tree.command(name="change_model", description="Change the AI model")
async def change_model_command(interaction: discord.Interaction):
    await interaction.response.defer()

    # Fetch available models that support content generation
    available_models = [
        m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods
    ]

    if not available_models:
        await interaction.followup.send("No models available for content generation.")
        return

    # Create a dropdown menu with the model names
    view = discord.ui.View()
    dropdown = discord.ui.Select(
        placeholder="Select a model",
        options=[discord.SelectOption(label=model) for model in available_models]
    )

    async def dropdown_callback(interaction: discord.Interaction):
        global model_name, model  # Access the global variables
        chosen_model = dropdown.values[0]
        model_name = chosen_model

        # Update the model with the selected model name
        model = genai.GenerativeModel(
            model_name=model_name,
            generation_config=text_generation_config,
            system_instruction=system_instruction,
            safety_settings=safety_settings,
            tools="code_execution"
        )

        await interaction.response.send_message(f"Model changed to: {chosen_model}", ephemeral=False)

    dropdown.callback = dropdown_callback
    view.add_item(dropdown)

    await interaction.followup.send("Choose a new model:", view=view, ephemeral=False)

def check_for_censorship(response):
  """
  Checks for censorship in the Gemini API response from chat.send_message.

  Args:
      response: The response object from chat.send_message.

  Returns:
      A tuple containing:
          - True if the response was censored, False otherwise.
          - The reason for censorship (if any).
  """
  candidates = response.get("candidates", [])
  if not candidates:
    return False, None  # No candidates generated, so no censorship

  # Check the first candidate (assuming you only want the top result)
  candidate = candidates[0]
  finish_reason = candidate.get("finish_reason")  # Note: 'finish_reason' instead of 'finishReason'
  safety_ratings = candidate.get("safety_ratings", [])

  if finish_reason == "SAFETY":
    # Censorship detected
    censorship_reasons = [
        f"{rating['category']}: {rating['probability']}" for rating in safety_ratings
    ]
    return True, ", ".join(censorship_reasons)

  return False, None

async def process_message(message: Message, is_webhook: bool = False) -> str:
    global chat_history, history

    utc_now = datetime.utcnow()
    spc_timezone = pytz.timezone('Etc/GMT')
    spc_now = utc_now.replace(tzinfo=pytz.utc).astimezone(spc_timezone)
    timestamp = spc_now.strftime('%Y-%m-%d %H:%M:%S')

    is_dm = isinstance(message.channel, discord.DMChannel)
    id_str = str(message.author.id) if is_dm else str(message.channel.id)

    # Check if the message is a reply to another message
    if message.reference and message.reference.message_id and is_webhook:
        # Fetch the original message being replied to
        replied_message = await message.channel.fetch_message(message.reference.message_id)
        bot_id = str(replied_message.author.id)  # Use the replied-to user's ID
    else:
        bot_id = "main_bot_" # Fallback to the current message author's ID

    username: str = str(message.author)
    user_message_with_timestamp = f"{timestamp} - ({username},[{message.author.id}]): {message.content}"

    print(f"({id_str}): {user_message_with_timestamp}")
    if is_webhook:
      system_instruction = load_webhook_system_instruction(bot_id, webhooks_path=webhooks_path)
      custom_model = genai.GenerativeModel(
         model_name=model_name,
         generation_config=text_generation_config,
         system_instruction=system_instruction,
         safety_settings=safety_settings,
         tools="code_execution"
         )
    else:
        custom_model = model
    history = load_chat_history(id_str, custom_path, bot_id)
    chat_history = check_expired_files(id_str, time_file_path, history, bot_id)
    chat = custom_model.start_chat(history=chat_history)
    #adding
    #loaded the chat for the main bot and the webhook

    url_pattern = re.compile(r'(https?://[^\s]+)')
    urls = url_pattern.findall(message.content)
    Direct_upload = False
    Link_upload = False
    attach_url = None

    if urls:
        attach_url = urls[0]
        Link_upload = True

    if message.attachments:
        for attachment in message.attachments:
            attach_url = attachment.url
            Direct_upload = True
            break

    try:
        if Direct_upload or Link_upload:
            format, downloaded_file = download_file(attach_url, id_str)

            if format in ('image/gif'):
                gif_clip = mp.VideoFileClip(downloaded_file)
                output_path = f"{downloaded_file.rsplit('.', 1)[0]}.mp4"
                gif_clip.write_videofile(output_path, codec='libx264')
                downloaded_file = output_path
                format = 'video/mp4'

            media_file = [upload_to_gemini(f"{downloaded_file}", mime_type=f"{format}"),]

            wait_for_files_active(media_file)

            save_filetwo(id_str, time_file_path, media_file[0].uri, bot_id)

            response = chat.send_message([user_message_with_timestamp, media_file[0]])
            response = response.text

            save_chat_history(id_str, chat, custom_path, bot_id)
            print(f"Bot: {response}")
            Direct_upload = False
            Link_upload = False

        else:
            response = chat.send_message(user_message_with_timestamp)
            response = response.text
            #print(chat)
            #is_censored, censorship_reason = check_for_censorship(response)
            """if is_censored:
                print(f"The response was censored due to: {censorship_reason}")
            else:
                print("The response was not censored.")"""
            save_chat_history(id_str, chat, custom_path, bot_id)
            print(f"Bot: {response}")

        return response

    except GoogleAPIError as e:
        error_message = await error_handler.handle_error(message, e, id_str)
        return error_message

@bot.event
async def on_message(message: Message) -> None:
    if message.author == bot.user:
        return

    is_webhook_interaction = False
    webhook = None

    try:
        # Check if the message is a reply to a webhook or mentions a webhook
        if message.reference:
            referenced_message = await message.channel.fetch_message(message.reference.message_id)
            if referenced_message.webhook_id:
                webhook = await bot.fetch_webhook(referenced_message.webhook_id)
                is_webhook_interaction = True

        if is_webhook_interaction and webhook:
            await message.add_reaction('🔴')
            response = await process_message(message, is_webhook=True)
            await send_message_webhook(webhook=webhook, response=response)
            await message.remove_reaction('🔴', bot.user)
        elif bot.user in message.mentions or isinstance(message.channel, discord.DMChannel):
            await message.add_reaction('🔴')
            try:
                # Force cache update by fetching the message again
                message = await message.channel.fetch_message(message.id)
                response = await process_message(message)
                await send_message_main_bot(message=message, response=response)
            except discord.NotFound:
                print(f"Message not found (even after fetching): {message.id}")
            except Exception as e:
                print(f"Error processing message {message.id}: {str(e)}")

            await message.remove_reaction('🔴', bot.user)

        # Process other bot commands
        await bot.process_commands(message)

    except discord.NotFound:
        print(f"Webhook or message not found for message {message.id}")
    except discord.Forbidden:
        print(f"Bot doesn't have permission to interact with webhook for message {message.id}")
    except Exception as e:
        print(f"Error processing message {message.id}: {str(e)}")

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user} (ID: {bot.user.id})')
    print('------')

    # Load existing webhooks from JSON
    json_file = os.path.join(webhooks_path, "webhooks_data.json")
    if os.path.exists(json_file):
        with open(json_file, "r") as f:
            webhooks_dict = json.load(f)

        # Fetch and store each webhook
        for webhook_id, data in webhooks_dict.items():
            try:
                webhook = await bot.fetch_webhook(int(webhook_id))
                webhooks[webhook.id] = webhook
            except discord.NotFound:
                print(f"Webhook {webhook_id} not found, removing from JSON")
                del webhooks_dict[webhook_id]

        # Save the updated dictionary back to the JSON file
        with open(json_file, "w") as f:
            json.dump(webhooks_dict, f, indent=4)

    await bot.tree.sync()

# STEP 5: MAIN ENTRY POINT
async def main() -> None:
    await bot.start(TOKEN)

# To run in a cloud service like Google Colab, Kaggle, etc.
nest_asyncio.apply()

# Run the main function
if __name__ == "__main__":
    await main()

Logged in as Mana Nagase#8027 (ID: 1228578114582482955)
------


ERROR:discord.client:Ignoring exception in on_ready
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/discord/client.py", line 449, in _run_event
    await coro(*args, **kwargs)
  File "<ipython-input-7-aaa081033730>", line 542, in on_ready
    for webhook_id, data in webhooks_dict.items():
RuntimeError: dictionary changed size during iteration


Webhook 1286037871563898923 not found, removing from JSON
(1277495950570553448): 2024-09-22 12:14:48 - (humanotbot,[738618866699075595]): Hry
Bot: *Mana beams back with a cheerful grin.*  "Hry to you too!  <:wavesm:1270658223632175114>  What's on your mind today?" 

Sending message via main bot...
(1277495950570553448): 2024-09-22 12:15:07 - (humanotbot,[738618866699075595]): Nothing much...
Bot: "Nothing much at all?  A quiet day can be nice.  Are you enjoying the peace and quiet, or are you feeling a little restless? 🤔" 

Sending message via main bot...
Sending message via webhook...
(1277495950570553448): 2024-09-22 12:17:13 - (humanotbot,[738618866699075595]): Uh


ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2076.41ms


Sending message via main bot...
Error: ⚠️ Error 500: Internal Server Error. Please try again later or reduce the input context. (Details: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting)
Sending message via webhook...
Error processing message 1287387227902705684: object of type 'NoneType' has no len()
Sending message via webhook...
(1277495950570553448): 2024-09-22 12:19:59 - (humanotbot,[738618866699075595]): Hey
Bot: "O-oh, um, hi," you stammer, caught off guard by her sudden appearance and delicate beauty. "I'm [your name]," you manage, trying to maintain eye contact without seeming creepy. 

You're very aware of her eyes on you as you speak, and you find yourself oddly flustered.  You try to focus on her question, pushing away any stray thoughts about how good she smells. "It'

For viewing /editing the chat_history


1. Enter the user/channel id(""THE FULLPATH IS BASED ON THE DEFAULT BEHAVIOUR OF THE PROGRAM, IF YOU HAVE CHANGED IT THEN CHANGE THE FILE  PATH ACCORDINGLY**)
2. Open the file using the fullpath
3. Run the Styling cell
4. Print the chat history by running the "print_chat_history_nicely(chat_history)"
You can view all the messages in this window You can see some numbers associated with them.
5. To modify a message enter the number associated with the message you want to modify.
A text box will appear where you can entre your new message and click on "sumbit" when done. (The new message would be visible in the new window that was created)
6. Now run the cell [Save back the chat history] and a message will tell your history is saved succesfully




In [ ]:
#@title Type the user/channel id
id = 1264073938175004757 # @param {type:"integer"}
bot_id = "" # @param {type:"string"} #for the main bot the bot_id = is main_bot_
fullpath = f"/content/drive/MyDrive/Discord_bot/{id}_chat_history.pkl"

In [ ]:
#@title Opens the chat history
with open(fullpath, 'rb') as file:
    chat_history = pickle.load(file)

In [ ]:
# @title Styling
"""from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import json"""

# Define the custom CSS once
custom_css = """
<style>
  .chat-container {
    display: flex;
    flex-direction: column;
    width: 100%;
    padding: 10px;
    border-radius: 5px;
    background-color: #000000;
    margin-top: 10px;
    box-sizing: border-box;
  }

  .chat-message {
    display: flex;
    align-items: flex-start;
    margin-bottom: 5px;
    border-bottom: 1px solid #FFFFFF;
    width: 100%;
    box-sizing: border-box;
  }

  .message-serial {
    font-weight: bold;
    margin-right: 5px;
    width: 30px;
    flex-shrink: 0;
  }

  .message-role {
    font-weight: bold;
    margin-right: 5px;
    width: 80px;
    flex-shrink: 0;
  }

  .message-text {
    flex: 1;
    word-wrap: break-word;
    white-space: pre-wrap;
  }

  .auto-resize-textarea {
    resize: none;
    overflow-y: hidden;
    min-height: 50px;
    max-height: 300px;
  }
</style>

<script>
function autoResize(textarea) {
    textarea.style.height = 'auto';
    textarea.style.height = textarea.scrollHeight + 'px';
}
</script>
"""

def apply_custom_css():
    """Applies the custom CSS and JavaScript."""
    display(HTML(custom_css))

def print_chat_history_nicely(chat_history):
    """Prints chat history using Colab widgets and custom CSS for enhanced formatting,
    including serial numbers for each message.

    Args:
      chat_history: A list of content objects representing the chat history.
    """
    clear_output(wait=True)
    apply_custom_css()

    chat_container_html = "<div class='chat-container'>"

    message_count = 0
    for content_obj in chat_history:
        if isinstance(content_obj, content.Content):
            message_role = content_obj.role
            text_content = content_obj.parts[0].text

            chat_container_html += f"""
            <div class="chat-message">
              <span class="message-serial">{message_count}. </span>
              <span class="message-role">{message_role}:</span>
              <span class="message-text">{text_content}</span>
            </div>
            """
            message_count += 1
        else:
            print("Unexpected content type found in chat history.")

    chat_container_html += "</div>"
    display(HTML(chat_container_html))

    print("Finished printing chat history.")

def display_and_replace_message(content_obj):
    """Displays a selected chat message, takes user input, and replaces the message with the input.
    Also includes a button to copy the displayed message.

    Args:
        content_obj: A content object representing a single chat message.
    """
    if isinstance(content_obj, content.Content):
        text_content = content_obj.parts[0].text
        message_role = content_obj.role

        input_text = widgets.Textarea(
            value='',
            placeholder='Type your message here...',
            description='New Message:',
            layout=widgets.Layout(width='100%', min_height='50px')
        )

        submit_button = widgets.Button(description="Submit")
        copy_button = widgets.Button(description="Copy Message")

        def update_display():
            clear_output(wait=True)
            apply_custom_css()

            message_element = f"""
            <div class="chat-container">
              <div class="chat-message" id="selected-message">
                <span class="message-role">{message_role}:</span>
                <span class="message-text">{text_content}</span>
              </div>
            </div>
            """
            display(HTML(message_element))
            display(widgets.HBox([copy_button, submit_button]))
            display(input_text)

            # Add JavaScript to make the textarea auto-resize
            display(HTML("""
            <script>
            var textarea = document.querySelector('.widget-textarea textarea');
            textarea.classList.add('auto-resize-textarea');
            textarea.setAttribute('onInput', 'autoResize(this)');
            </script>
            """))

        def on_submit_button_clicked(b):
            nonlocal text_content
            new_text_content = input_text.value
            content_obj.parts[0].text = new_text_content
            text_content = new_text_content
            update_display()

        def on_copy_button_clicked(b):
            js_code = f"""
            var textarea = document.createElement('textarea');
            textarea.value = {json.dumps(text_content)};
            document.body.appendChild(textarea);
            textarea.select();
            document.execCommand('copy');
            document.body.removeChild(textarea);
            """
            display(HTML(f"<script>{js_code}</script>"))
            print("Message copied to clipboard!")

        submit_button.on_click(on_submit_button_clicked)
        copy_button.on_click(on_copy_button_clicked)
        update_display()

    else:
        print("Unexpected content type at the specified index.")

# Example usage:
# print_chat_history_nicely(chat_history)
# display_and_replace_message(chat_history[specific_index])

In [ ]:
#@title Prints the chat history on the screen
print_chat_history_nicely(chat_history)

In [ ]:
#@markdown Replace message
selected_message = 6 # @param {type:"integer"}
display_and_replace_message(chat_history[selected_message])

Textarea(value='Red', description='New Message:', layout=Layout(min_height='50px', width='100%'), placeholder=…

##**Save back the chat history**

In [ ]:
with open(fullpath, 'wb') as file:
    pickle.dump(chat_history, file)
    print("Successfully updated the chat_history.pkl")

Successfully updated the chat_history.pkl
